# 자연어 처리 (토크나이징)

## 자연어 처리 함수 레벨1

In [36]:
import re
from konlpy.tag import Okt
from kiwipiepy import Kiwi
import pandas as pd

In [37]:
okt = Okt()

def get_word_list_with_okt(series, stopwords = [], pos_tags = []):
    word_list = []
    
    for i, text in enumerate(series.tolist()):
        # STEP1: 데이터 전처리
        new_text = re.sub("[^a-zA-Z가-힣\\s]", "", text)

        # STEP2: 형태소 분석
        result = okt.pos(new_text)

        # STEP3: 조건에 맞는 단어 담기
        for word, pos in result:
            if len(word) > 1 and word not in stopwords:
                if pos_tags == []:                
                    word_list.append(word)
                elif pos in pos_tags:
                    word_list.append(word)
        
    return word_list

## 자연어 처리 함수 레벨2

In [38]:
okt = Okt()

def get_word_list(series, nlp_type = "okt", stopwords = [], pos_tags = []):
    word_list = []
    
    nlp = Okt()
    if nlp_type == 'kiwi':
        nlp = Kiwi()

    for i, text in enumerate(series.tolist()):
        # STEP1: 데이터 전처리
        new_text = re.sub("[^a-zA-Z가-힣\\s]", "", text)

        # STEP2: 형태소 분석
        result = nlp.pos(new_text)

        # STEP3: 조건에 맞는 단어 담기
        for word, pos in result:
            if len(word) > 1 and word not in stopwords:
                if pos_tags == []:                
                    word_list.append(word)
                elif pos in pos_tags:
                    word_list.append(word)
        
    return word_list

## 자연어 처리 함수 레벨3

In [ ]:
def get_token_list(
    series,
    nlp_type = "okt",
    stopwords = [],
    pos_tags = None,
    min_len = 2,
    flatten = True
):
    """
    텍스트 시리즈에서 전처리 → 형태소 분석 → (불용어, 품사, 길이) 필터 → 토큰 리스트 반환.

    Args:
        series: 문자열 iterable (예: df['text'])
        nlp_type: "okt" 또는 "kiwi"
        stopwords: 제외할 단어들
        pos_tags: 포함할 품사 태그 목록 (None/빈 리스트면 전 품사 허용)
            - OKT 예: ["Noun","Adjective"]
            - Kiwi 예: ["NNG","NNP","VA"]
            - 한국어 품사 태그 비교표: https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit?gid=0#gid=0
        min_len: 최소 글자 수 필터
        flatten: True면 모든 리뷰 토큰을 하나의 리스트로 병합, False면 리뷰별 리스트 유지

    Returns:
        - flatten=True  -> List[str]
        - flatten=False -> List[List[str]]
    """
    word_list = []
    
    nlp = Okt()
    if nlp_type == 'kiwi':
        nlp = Kiwi()
    
    for i, text in enumerate(series.tolist()):
        # STEP1: 데이터 전처리
        new_text = re.sub("[^a-zA-Z가-힣\\s]", "", text)

        # STEP2: 형태소 분석
        result = nlp.pos(new_text)
    
        temp_list = []
        # STEP3: 조건에 맞는 단어 담기
        for word, pos in result:
            # word의 길이가 min_len 보다 짧거나 불용어 포함시 넘어감
            if len(word) < min_len or word in stopwords:
                continue

            if pos_tags == None:                
                temp_list.append(word)
            elif pos in pos_tags:
                temp_list.append(word)
        
        if flatten:
            word_list.extend(temp_list)
        else:
            word_list.append(temp_list)
        
    return word_list

## 코드

In [40]:
data = pd.read_csv("data/raw/구글플레이스토어_땡겨요_리뷰.csv", index_col=0)
result = get_token_list(data["content"], nlp_type='okt', pos_tags = ['Noun'], flatten=False)

# csv파일로 저장하기
data['tokens'] = result
data.to_csv("data/processed/구글플레이스토어_땡겨요_리뷰_토크나이징.csv")

In [44]:
# main.py
from utils.io_util import save_text_to_file
from utils.list_util import flatten_2d_list

# 저장
save_text_to_file(flatten_2d_list(result), "data/words/구글플레이스토어_땡겨요_리뷰_토크나이징.txt")